In [1]:
import pandas as pd
%pip install jsonlines
import jsonlines as jl
import matplotlib.pyplot as plt
%pip install openpyxl
%pip install xlrd
%pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize
import string
import re



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
train_path = '1_Data/Raw_Data/train.jsonl'
test_path = '1_Data/Raw_Data/test.jsonl'
dev_path = '1_Data/Raw_Data/dev.jsonl'

def read_jsonl_to_df(filepath):
    data = []
    with jl.open(filepath) as reader:
        for obj in reader:
            data.append(obj)
    return pd.DataFrame(data)

#create DataFrames
train_df_old = read_jsonl_to_df(train_path)
train_df = read_jsonl_to_df(train_path)

test_df = read_jsonl_to_df(test_path)
dev_df = read_jsonl_to_df(dev_path)
add_df = pd.read_excel('1_Data/Raw_Data/additional_data.xls')


### Preprocessing

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


def data_preprocess(text):
    #lowercase the text
    text = text.lower()     
    # (2.2) Tokenize the text into words
    tokens = word_tokenize(text)   
    new_tokens = []
    #remove stopwords
    lemmatizer = WordNetLemmatizer()
    for w in tokens:
        if ( w not in stopwords.words('english') and w not in string.punctuation ) and not drop_word(w):
        #if not drop_word(w):
            #lemmatize the words
            lemma_token = lemmatizer.lemmatize(w)
            new_tokens.append(lemma_token)
            
    return new_tokens

[nltk_data] Downloading package punkt to /Users/jonathan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonathan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jonathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jonathan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Drop words in special cases

In [4]:

special_drop = ['\"', '']

def drop_word(word):
    #regex patterns
    single_quote_pattern = r"^'"
    non_alnum_pattern = r"^[^a-zA-Z0-9]+$"
    
    #check: start with quote, only non-alpha-numeric, or in special_drop
    if re.match(single_quote_pattern, word) or re.match(non_alnum_pattern, word) or word in special_drop:
        return True
    else:
        return False


In [5]:
train_df["Preprocessed"] = train_df["text"].apply(data_preprocess)
test_df["Preprocessed"] = test_df["text"].apply(data_preprocess)
dev_df["Preprocessed"] = dev_df["text"].apply(data_preprocess)

train_df.to_csv('1_Data/Raw_Data/train_preprocessed.csv', index=False)
train_df_old.to_csv('1_Data/Raw_Data/train_old.csv', index=False)
test_df.to_csv('1_Data/Raw_Data/test_preprocessed.csv', index=False)
dev_df.to_csv('1_Data/Raw_Data/dev_preprocessed.csv', index=False)



In [6]:
#check data

clim = train_df[train_df["label"] == 1].reset_index(drop=True)


for i in range(5):
    print(clim["Preprocessed"][i])
    print(clim["text"][i])

type(train_df["Preprocessed"][0])

['project', 'make', 'significant', 'contribution', 'german', 'european', 'hydrogen', 'strategy', 'hence', 'achievement', 'climate', 'target']
The project will make a significant contribution to the German and European hydrogen strategy and hence to achievement of the climate targets.
['environmentally', 'preferable', 'purchasing', 'guideline', 'formulated', 'enforce', 'green', 'purchasing', 'apply', 'part', 'raw', 'material', 'also', 'packaging', 'material']
Our Environmentally Preferable Purchasing Guidelines, which we have formulated to enforce “green” purchasing, apply not only to parts and raw materials but also to packaging materials.
['global', 'leader', 'insuring', 'low-carbon', 'technology', 'provide', 'standardized', 'tailor-made', 'insurance', 'product', 'part', 'sustainable', 'solution', 'see', 'section', '03.4']
As a global leader in insuring low-carbon technologies, we provide standardized and tailor-made insurance products as part of our Sustainable Solutions (see section

list

# Check for Nan Values

In [7]:
#Check for NaN values
print("Train Data:")
print(train_df.isnull().sum())

print("\nTest Data:")
print(test_df.isnull().sum())

print("\nDev Data:")
print(dev_df.isnull().sum())

Train Data:
text            0
label           0
Preprocessed    0
dtype: int64

Test Data:
text            0
label           0
Preprocessed    0
dtype: int64

Dev Data:
text            0
label           0
Preprocessed    0
dtype: int64
